In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [5]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.externals import joblib

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


**Extracting music and features**

**Dataset**

We use GTZAN genre collection dataset for classification.

The dataset consists of 10 genres 

1.   Blues
2.   Rock
3.Classical
4.Country
5.Disco
6.Hiphop
7.Jazz
8.Metal
9.Pop
10.Reggae

Each genre contains 100 songs. Total dataset: 1000 songs

In [6]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'Blues Classical Country Disco Hiphop Jazz Metal Pop Reggae Rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'GTZAN/{g}'):
        songname = f'GTZAN/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .We can now easily extract features from them

**Extracting features from Spectrogram**
 
 We will extract

Mel-frequency cepstral coefficients (MFCC)(20 in number)

Spectral Centroid,

Zero Crossing Rate

Chroma Frequencies

Spectral Roll-off.

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [8]:
header

['filename',
 'chroma_stft',
 'rmse',
 'spectral_centroid',
 'spectral_bandwidth',
 'rolloff',
 'zero_crossing_rate',
 'mfcc1',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc14',
 'mfcc15',
 'mfcc16',
 'mfcc17',
 'mfcc18',
 'mfcc19',
 'mfcc20',
 'label']

In [0]:
#writing the data and coeficiwn into a csv file

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'Blues Classical Country Disco Hiphop Jazz Metal Pop Reggae Rock'.split()
for g in genres:
    for filename in os.listdir(f'GTZAN/{g}'):
        songname = f'GTZAN/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

**Analysing data in pandas**

In [17]:
data = pd.read_csv('data.csv')
data.head()


,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
Blues,blues.00086.wav,0.365865,0.124991,1245.609395,1634.415029,2517.684776,0.048827,-212.790368,139.057366,9.459221,...,2.640718,-1.025016,-1.406932,-4.704802,1.138613,-9.472101,-2.187314,-4.100537,-3.352784,Blues
Blues,blues.00092.wav,0.391010,0.063085,918.405453,1437.609189,1599.673660,0.032318,-332.482336,128.042922,28.936692,...,-0.424217,1.495610,-1.555774,-0.837087,2.156455,-3.899823,-4.198857,-5.930102,-3.434429,Blues
Blues,blues.00079.wav,0.421495,0.270870,2443.298208,2358.443293,5267.601481,0.124735,-3.528043,92.914318,-25.110401,...,14.742995,-15.600666,9.548172,-14.788735,2.067856,-13.821133,3.563616,-5.965995,-1.649998,Blues
Blues,blues.00045.wav,0.429511,0.148366,1739.019621,2290.401739,4551.155813,0.048052,-163.211141,89.791041,12.979256,...,6.008639,-2.910059,-0.434613,-17.545474,4.154533,-5.990226,-7.860423,-8.565096,1.857711,Blues
Blues,blues.00051.wav,0.393756,0.196723,1977.172377,1927.803692,3942.834492,0.106627,-55.579243,114.935848,-37.052831,...,12.782317,-16.528681,3.793788,-7.890870,8.477609,-4.065210,3.207441,-5.178250,-1.279524,Blues


In [18]:
data.shape

(1000, 28)

In [0]:
# Dropping unneccesary columns
#data = data.drop(['filename'],axis=1)

In [23]:
data.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
Blues,0.365865,0.124991,1245.609395,1634.415029,2517.684776,0.048827,-212.790368,139.057366,9.459221,35.441294,...,2.640718,-1.025016,-1.406932,-4.704802,1.138613,-9.472101,-2.187314,-4.100537,-3.352784,Blues
Blues,0.391010,0.063085,918.405453,1437.609189,1599.673660,0.032318,-332.482336,128.042922,28.936692,41.828353,...,-0.424217,1.495610,-1.555774,-0.837087,2.156455,-3.899823,-4.198857,-5.930102,-3.434429,Blues
Blues,0.421495,0.270870,2443.298208,2358.443293,5267.601481,0.124735,-3.528043,92.914318,-25.110401,45.177700,...,14.742995,-15.600666,9.548172,-14.788735,2.067856,-13.821133,3.563616,-5.965995,-1.649998,Blues
Blues,0.429511,0.148366,1739.019621,2290.401739,4551.155813,0.048052,-163.211141,89.791041,12.979256,80.691273,...,6.008639,-2.910059,-0.434613,-17.545474,4.154533,-5.990226,-7.860423,-8.565096,1.857711,Blues
Blues,0.393756,0.196723,1977.172377,1927.803692,3942.834492,0.106627,-55.579243,114.935848,-37.052831,64.896513,...,12.782317,-16.528681,3.793788,-7.890870,8.477609,-4.065210,3.207441,-5.178250,-1.279524,Blues


**encoding labels**

In [0]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [25]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

**Scaling the Feature columns**

In [0]:
#all data goes in range of -1 to 1
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [30]:
X

array([[-0.15678386, -0.09044175, -1.33624993, ..., -0.69679577,
        -0.47200345, -0.58866528],
       [ 0.15112013, -1.03338156, -1.79349203, ..., -1.216957  ,
        -0.95940713, -0.60995113],
       [ 0.52440978,  2.13155555,  0.33742716, ...,  0.79032646,
        -0.96896914, -0.14472764],
       ...,
       [-0.43431617, -0.11431384, -0.92889577, ...,  0.1721994 ,
         0.75116773, -1.23431979],
       [-0.02312941,  1.10972212, -0.92177934, ..., -1.48705433,
        -0.76994196, -1.35175137],
       [ 0.78379404,  0.20460947,  0.49942812, ..., -1.12653082,
        -1.01627317, -1.11563051]])

In [31]:
pd.DataFrame(X, columns=data.columns[:-1]).head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,-0.156784,-0.090442,-1.336250,-1.156005,-1.304981,-1.310835,-0.681847,1.261492,0.847677,-0.051131,...,0.907894,-0.272699,0.611595,-0.636806,-0.171291,-0.002048,-1.210250,-0.696796,-0.472003,-0.588665
1,0.151120,-1.033382,-1.793492,-1.530107,-1.888222,-1.705649,-1.876550,0.909775,1.745912,0.332277,...,0.757399,-0.729201,1.020268,-0.666533,0.622577,0.220342,0.014864,-1.216957,-0.959407,-0.609951
2,0.524410,2.131556,0.337427,0.220280,0.442126,0.504578,1.406901,-0.211963,-0.746556,0.533335,...,-1.302685,1.529858,-1.751577,1.551158,-2.241070,0.200984,-2.166423,0.790326,-0.968969,-0.144728
3,0.622571,0.265596,-0.646747,0.090942,-0.013054,-1.329363,-0.186973,-0.311696,1.010009,2.665174,...,-2.132052,0.228931,0.305970,-0.442614,-2.806904,0.656907,-0.444729,-2.163795,-1.661380,0.769776
4,0.184741,1.002162,-0.313947,-0.598311,-0.399539,0.071504,0.887353,0.491236,-1.297301,1.717033,...,0.055711,1.237827,-1.902038,0.401887,-0.825248,1.601466,-0.021498,0.698224,-0.759111,-0.048140


In [32]:
# Saving our Scalers and Encoders for testing purposes at real time

from sklearn.externals import joblib

encoder_filename = "encoder.save"
joblib.dump(encoder, encoder_filename) 

scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

['scaler.save']

**Dividing data into training and Testing set¶**


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [34]:
len(y_train)

800

In [35]:
len(y_test)

200

In [36]:
X_train[10]

array([ 0.6307393 ,  1.6916627 ,  0.2852088 ,  0.04990495,  0.26427191,
        0.38492789,  1.16657802, -0.16562778, -0.53739424,  0.81461821,
       -1.10913716,  0.77978586, -1.25449947,  1.33006392, -1.07196282,
        1.09540901, -0.67450001,  1.40098144, -0.91934484,  1.27971622,
       -1.32035743,  0.84443393, -0.38982525,  2.53567715,  1.59146321,
        2.69304023])

In [37]:
X_train.shape[1]

26

**Building our Network**

In [38]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))


model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6912      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 65,226
Trainable params: 65,226
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [42]:
 model.fit(X_train,
                    y_train,
                    epochs=25,
                    batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
800/800 [==============================] - 1s 921us/step - loss: 2.2202 - acc: 0.2375
Epoch 2/25
800/800 [==============================] - 0s 41us/step - loss: 1.9230 - acc: 0.3987
Epoch 3/25
800/800 [==============================] - 0s 39us/step - loss: 1.6406 - acc: 0.4025
Epoch 4/25
800/800 [==============================] - 0s 41us/step - loss: 1.4294 - acc: 0.5012
Epoch 5/25
800/800 [==============================] - 0s 41us/step - loss: 1.2622 - acc: 0.5413
Epoch 6/25
800/800 [==============================] - 0s 49us/step - loss: 1.1458 - acc: 0.6088
Epoch 7/25
800/800 [==============================] - 0s 52us/step - loss: 1.0678 - acc: 0.6212
Epoch 8/25
800/800 [==============================] - 0s 41us/step - loss: 0.9899 - acc: 0.6525
Epoch 9/25
800/800 [==============================] - 0s 40us/step - loss: 0.9304 - acc: 0.6638
Epoch 10/25
800/800 [==============================] - 0s 42us/step - loss: 0.8675 - ac

In [43]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 231us/step


In [44]:
print('test_acc: ',test_acc)


test_acc:  0.605


**Validating on training data to see if its overfitting**

Let's set apart 200 samples in our training data to use as a validation set:

In [0]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]


In [46]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=35,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/35
600/600 [==============================] - 0s 583us/step - loss: 2.3001 - acc: 0.0800 - val_loss: 2.1562 - val_acc: 0.3450
Epoch 2/35
600/600 [==============================] - 0s 66us/step - loss: 2.1616 - acc: 0.2767 - val_loss: 2.0370 - val_acc: 0.3500
Epoch 3/35
600/600 [==============================] - 0s 61us/step - loss: 2.0299 - acc: 0.3200 - val_loss: 1.8995 - val_acc: 0.4200
Epoch 4/35
600/600 [==============================] - 0s 60us/step - loss: 1.8837 - acc: 0.3850 - val_loss: 1.7779 - val_acc: 0.4200
Epoch 5/35
600/600 [==============================] - 0s 65us/step - loss: 1.7528 - acc: 0.3950 - val_loss: 1.6900 - val_acc: 0.4100
Epoch 6/35
600/600 [==============================] - 0s 63us/step - loss: 1.6341 - acc: 0.4183 - val_loss: 1.6175 - val_acc: 0.4550
Epoch 7/35
600/600 [==============================] - 0s 72us/step - loss: 1.5225 - acc: 0.4767 - val_loss: 1.5517 - val_acc: 0.4650
Epoch 8/35
600/600 [==

In [47]:
results

[1.2059508991241454, 0.615]

**Predictions on Test Data**

In [0]:
predictions = model.predict(X_test)

In [49]:
predictions[0].shape

(10,)

In [50]:
predictions

array([[3.4304458e-06, 5.3434178e-08, 1.2146849e-03, ..., 9.9588341e-01,
        1.9556477e-04, 3.6123014e-04],
       [8.0433828e-01, 4.4041825e-04, 3.4245164e-03, ..., 1.8555127e-04,
        1.0812747e-03, 1.0502500e-04],
       [3.8549680e-02, 7.4440140e-07, 7.5844279e-04, ..., 1.4492992e-06,
        4.2090024e-04, 5.2166224e-01],
       ...,
       [3.4428303e-04, 9.7017015e-05, 2.1349373e-01, ..., 4.8384988e-01,
        1.5707662e-02, 1.3348323e-01],
       [4.7110502e-06, 1.3855490e-07, 5.4178997e-03, ..., 9.9132746e-01,
        4.2233733e-04, 4.7626684e-04],
       [1.3012140e-03, 6.8534875e-01, 4.0170904e-03, ..., 3.7819216e-05,
        2.7484325e-04, 1.1298406e-05]], dtype=float32)

In [53]:
np.sum(predictions[0])

1.0

In [55]:
predictions[0]

array([3.4304458e-06, 5.3434178e-08, 1.2146849e-03, 1.3558208e-03,
       6.9763727e-04, 2.8559228e-04, 2.6026469e-06, 9.9588341e-01,
       1.9556477e-04, 3.6123014e-04], dtype=float32)

In [57]:
encoder.inverse_transform(model.predict_classes(np.array([X_test[0],])))

array(['Pop'], dtype=object)

In [58]:
np.array([X_test[0],])


array([[ 0.64590752,  1.58570534,  1.82917135,  1.72253825,  1.70613123,
         1.63575813,  1.01631114, -1.42613255,  1.19855734, -0.99201066,
         0.65503078, -1.00154146,  1.70563751, -0.8524481 ,  1.80364103,
        -0.83088466,  0.28559021, -0.64618792,  0.0907494 , -1.24582211,
         0.2786972 , -0.88269719,  1.00878906,  0.101571  ,  1.23561907,
         0.78768795]])

In [0]:
model.save('music_classifier.h5')